In [49]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [50]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import powerlaw
from sklearn.metrics import silhouette_score
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from sklearn.cluster import DBSCAN
from sklearn.metrics.pairwise import pairwise_distances
from tslearn.metrics import dtw
from tslearn.clustering import KShape
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.cluster.hierarchy import fcluster
from tslearn.metrics import dtw
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from collections import Counter
import random
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist
from matplotlib.font_manager import FontProperties
import matplotlib.lines as mlines
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.colors as mcolors
import matplotlib.cm as cm
rc('font',**{'family':'serif','serif':['Times New Roman']})
rc('text', usetex=True)
%matplotlib inline
#Powerlaw: A Python Package for Analysis of Heavy-Tailed Distributions
# Initialize an empty list to store the data
data = []

# Open the text file for reading with the specified encoding
with open(r'C:\Users\oscar\OneDrive - Universidad Nacional de Colombia\Tesis\Comportamineto humano\Check-in-NY\dataset_tsmc2014\dataset_TSMC2014_NYC.txt', 'r', encoding='latin-1') as file:
    # Read each line in the file
    for line in file:
        # Split the line into columns based on the tab delimiter
        columns = line.strip().split('\t')
        
        # Append the columns to the data list
        data.append(columns)

# Define the column names
column_names = ['User ID', 'Venue ID', 'Venue category ID', 'Venue category name', 'Latitude', 'Longitude', 'Timezone offset in minutes', 'UTC time']

# Create a DataFrame from the data list
df = pd.DataFrame(data, columns=column_names)

# Optionally, convert columns to appropriate data types
df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)
df['Timezone offset in minutes'] = df['Timezone offset in minutes'].astype(int)

# Display the DataFrame
df.head()


,User ID,Venue ID,Venue category ID,Venue category name,Latitude,Longitude,Timezone offset in minutes,UTC time
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,Tue Apr 03 18:00:09 +0000 2012
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,Tue Apr 03 18:00:25 +0000 2012
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,Tue Apr 03 18:02:24 +0000 2012
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,Tue Apr 03 18:02:41 +0000 2012
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,Tue Apr 03 18:03:00 +0000 2012


In [51]:


# Normalización de latitud y longitud
scaler = MinMaxScaler()
df[['Latitude', 'Longitude']] = scaler.fit_transform(df[['Latitude', 'Longitude']])

# Especificación del formato de la marca de tiempo
date_format = "%a %b %d %H:%M:%S +0000 %Y"

# Conversión de la marca de tiempo a características útiles
df['UTC time'] = pd.to_datetime(df['UTC time'], format=date_format)
df['Hour'] = df['UTC time'].dt.hour
df['DayOfWeek'] = df['UTC time'].dt.dayofweek

# Agrupación por usuario
grouped = df.groupby('User ID')

# Definición de la secuencia de longitud n
sequence_length = 10
data = []

for user_id, group in grouped:
    group = group.sort_values(by='UTC time')  # Ordenar por tiempo
    features = group[['Latitude', 'Longitude', 'Hour', 'DayOfWeek']].values
    generator = TimeseriesGenerator(features, features, length=sequence_length, batch_size=1)
    for i in range(len(generator)):
        x, y = generator[i]
        data.append((x[0], y[0]))  # Asegurarse de extraer correctamente los datos

# Verificación de las dimensiones de los datos
print(f"Número de secuencias: {len(data)}")
print(f"Dimensiones de cada secuencia: {data[0][0].shape if data else 'N/A'}")
print(f"Dimensiones de cada etiqueta: {data[0][1].shape if data else 'N/A'}")

# Convertir la lista de secuencias a numpy arrays y asegurar la forma correcta
X = np.array([item[0] for item in data])
y = np.array([item[1] for item in data])

# Ajustar las dimensiones para que sean compatibles con el modelo LSTM
X = X.reshape(len(X), sequence_length, -1)  # Debería ser (n_samples, sequence_length, n_features)
y = y.reshape(len(y), -1)  # Debería ser (n_samples, n_features)

# Verificación de las formas de las entradas
print("Forma de X:", X.shape)  # Debería ser (n_samples, sequence_length, n_features)
print("Forma de y:", y.shape)  # Debería ser (n_samples, n_features)



ValueError: `start_index+length=100 > end_index=99` is disallowed, as no part of the sequence would be left to be used as current step.

In [ ]:
# Creación del modelo LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(sequence_length, X.shape[2])))
model.add(Dense(X.shape[2]))  # Salida con el mismo número de características que la entrada
model.compile(optimizer='adam', loss='mse')

# Entrenamiento del modelo
model.fit(X, y, epochs=10, verbose=1)

Epoch 1/10

6769/6769 [==============================] - 70s 10ms/step - loss: 12.1782
Epoch 2/10
6769/6769 [==============================] - 72s 11ms/step - loss: 11.7502
Epoch 3/10
6769/6769 [==============================] - 71s 10ms/step - loss: 11.6577
Epoch 4/10
6769/6769 [==============================] - 76s 11ms/step - loss: 11.6087
Epoch 5/10
6769/6769 [==============================] - 86s 13ms/step - loss: 11.5682
Epoch 6/10
6769/6769 [==============================] - 78s 12ms/step - loss: 11.5433
Epoch 7/10
6769/6769 [==============================] - 78s 11ms/step - loss: 11.5205
Epoch 8/10
6769/6769 [==============================] - 78s 12ms/step - loss: 11.5093
Epoch 9/10
6769/6769 [==============================] - 83s 12ms/step - loss: 11.4898
Epoch 10/10
6769/6769 [==============================] - 76s 11ms/step - loss: 13.2321


In [ ]:
def detect_anomalies(model, X, threshold=0.01):
    predictions = model.predict(X)
    mse = np.mean(np.power(X - np.expand_dims(predictions, axis=1), 2), axis=(1, 2))  # Ajuste en el cálculo del MSE
    anomalies = mse > threshold
    return anomalies, mse


# Detectar anomalías
anomalies, mse = detect_anomalies(model, X)

# Imprimir resultado
print("Anomalies:", anomalies)
print("MSE:", mse)

6769/6769 [==============================] - 48s 7ms/step
Anomalies: [ True  True  True ...  True  True  True]
MSE: [13.43465706 17.11985166 16.90360602 ... 11.06232316 24.14922832
 20.01994358]


In [ ]:
true_count = np.count_nonzero(anomalies)
false_count = len(anomalies) - true_count

print("Number of True values (anomalies):", true_count)
print("Number of False values (non-anomalies):", false_count)

Number of True values (anomalies): 216598
Number of False values (non-anomalies): 0


In [ ]:
df['column_name'] = df['column_name'].str.replace(r'^(0550|0570)', '', regex=True)

In [ ]:
# Group the dataframe by latitude and longitude
grouped = df.groupby(['Latitude', 'Longitude'])

# Count the number of occurrences and create a list of IDs
count = grouped.size().reset_index(name='Count')
ids = grouped['ID'].apply(list).reset_index(name='IDs')